# 1. 사전작업
setup.sh 업로드 후 실행해 주세요.

In [0]:
! bash setup.sh

Installing openjdk-8-jdk....
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk openjdk-8-jre x11-utils
0 upgraded, 8 newly installed, 0 to remove and 7 not upgraded.
Need to get 4,954 kB of archives.
After this operation, 13.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubun

# 2. 자연어처리의 기본

## Spacy를 사용하여 토큰을 분리하기

In [0]:
import spacy
chomsky = "Colorless green ideas sleep furiously"
spacy_en = spacy.load('en')

def tokenize_en(text):
  return [tok.text for tok in spacy_en.tokenizer(text)]
print(tokenize_en(chomsky))

AttributeError: ignored

## NLTK를 사용하여 토큰을 분리하기

In [0]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
print(word_tokenize(chomsky))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['Colorless', 'green', 'ideas', 'sleep', 'furiously']


## python의 기본 함수 split()을 이용하여 토큰을 분리하기

In [0]:
print(chomsky.split())

['Colorless', 'green', 'ideas', 'sleep', 'furiously']


## Konlpy로 한국어를 처리하기

In [0]:
from konlpy.tag import Mecab

kor_sentence = "펭수를 보아라. 행복해질 것이니"
tokenizer = Mecab()

tokenized_sent = tokenizer.pos(kor_sentence)
print(tokenized_sent)
print([(k, tokenizer.tagset.get(v)) for k, v in tokenized_sent])

[('펭', 'NNG'), ('수', 'NNG'), ('를', 'JKO'), ('보', 'VV'), ('아라', 'EF'), ('.', 'SF'), ('행복', 'NNG'), ('해질', 'XSA+EC+VX+ETM'), ('것', 'NNB'), ('이', 'VCP'), ('니', 'EC')]
[('펭', '일반 명사'), ('수', '일반 명사'), ('를', '목적격 조사'), ('보', '동사'), ('아라', '종결 어미'), ('.', '마침표, 물음표, 느낌표'), ('행복', '일반 명사'), ('해질', None), ('것', '의존 명사'), ('이', '긍정 지정사'), ('니', '연결 어미')]


## 단어장 만들기

In [0]:
with open("./nsmc/ratings.txt") as file:
    raw_data = file.read().splitlines()[1:]
    data = [line.split("\t")[1:] for line in raw_data]
    data = [(tokenizer.morphs(sent), int(label)) for (sent, label) in data]
    
sample_data = data[4:14] + data[-10:]
print(sample_data[1])

(['사랑', '을', '해', '본', '사람', '이', '라면', '처음', '부터', '끝', '까지', '웃', '을', '수', '있', '는', '영화'], 1)


In [0]:
def build_vocab(data):
    flatten = lambda d: [token for sent in d for token in sent] # 이중 리스트 구조를 하나의 리스트 구조로 만들어준다.
    vocab = {}
    vocab['<unk>'] = 0
    vocab['<pad>'] = 1
    for token in flatten(list(zip(*data))[0]):
        if vocab.get(token) is None:
            vocab.setdefault(token, len(vocab))
    return vocab

In [0]:
vocab = build_vocab(sample_data)
print('{}개의 단어를 단어장에 추가'.format(len(vocab)))

def show_vocab(vocab):
  i=0
  for key, value in vocab.items():
    if i >= 5:
      print("{}{} {}".format(value, ': ', key))
      i=0
    else:
      print("{}{}\t{}\t".format(value, ': ', key), end='')
      i += 1
show_vocab(vocab)

157개의 단어를 단어장에 추가
0: 	<unk>	1: 	<pad>	2: 	안개	3: 	자욱	4: 	한	5:  밤하늘
6: 	에	7: 	떠	8: 	있	9: 	는	10: 	초승달	11:  같
12: 	은	13: 	영화	14: 	.	15: 	사랑	16: 	을	17:  해
18: 	본	19: 	사람	20: 	이	21: 	라면	22: 	처음	23:  부터
24: 	끝	25: 	까지	26: 	웃	27: 	수	28: 	완전	29:  감동
30: 	입니다	31: 	다시	32: 	봐도	33: 	개	34: 	들	35:  의
36: 	전쟁	37: 	2	38: 	나오	39: 	나요	40: 	?	41:  면
42: 	1	43: 	빠	44: 	로	45: 	보	46: 	고	47:  싶
48: 	음	49: 	굿	50: 	바보	51: 	가	52: 	아니	53:  라
54: 	병	55: 	쉰	56: 	인	57: 	듯	58: 	내	59:  나이
60: 	와	61: 	를	62: 	지금	63: 	나	64: 	적	65:  다
66: 	하지만	67: 	훗날	68: 	보면대	69: 	사	70: 	하나	71:  그
72: 	감정	73: 	완벽	74: 	하	75: 	게	76: 	이해	77:  할
78: 	것	79: 	만	80: 	..	81: 	재밌	82: 	고질라	83:  니무
84: 	귀엽	85: 	능	86: 	ㅋㅋ	87: 	오페라	88: 	화	89:  라고
90: 	해야	91: 	작품	92: 	극단	93: 	평갈	94: 	림	95:  어쩔
96: 	없	97: 	장르	98: 	무협	99: 	인데	100: 	기	101:  엔
102: 	코믹	103: 	던데	104: 	막장	105: 	평점	106: 	점	107:  도
108: 	아깝	109: 	나치	110: 	입장	111: 	에서	112: 	갑자기	113:  연속
114: 	으로	115: 	네	116: 	뭔일	117: 	었	118: 	태권도	119:  ??
120: 	왜	121: 	봤	122: 	을까	123: 	예고편	124: 	-	125:  개연
126

In [0]:
def numericalize(sent, vocab):
    temp= []
    for token in sent:
        if vocab.get(token) is None:
            temp.append(vocab['<unk>'])
        else:
            temp.append(vocab.get(token))
    return temp

numerical_data = [(numericalize(sent, vocab), label) \
                  for sent, label in sample_data]
print(numerical_data[0])

([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 1)


In [0]:
print("패딩 이전의 문장 길이:")
print([len(sent) for sent, _ in numerical_data])
max_len = max([len(sent) for (sent, _) in numerical_data])

for sent, _ in numerical_data:
    if len(sent) < max_len:
        sent += [vocab['<pad>']] * (max_len - len(sent))
print("패딩 이후의 문장 길이: ")
print([len(sent) for sent, _ in numerical_data])
print("패딩 이후의 1번째 문장의 토큰들: ")
print(numerical_data[0])

패딩 이전의 문장 길이:
[13, 17, 6, 17, 1, 8, 38, 2, 6, 20, 19, 18, 3, 9, 11, 6, 5, 15, 15, 8]
패딩 이후의 문장 길이: 
[38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38]
패딩 이후의 1번째 문장의 토큰들: 
([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 1)


## TorchText 활용하기
1. 필드 지정
2. 데이터셋 만들기
3. 단어장 생성
4. 데이터로더 설정


### 1. 필드 지정

In [0]:
import torch
from torchtext.data import Field

TEXT = Field(sequential=True,
             use_vocab=True,
             tokenize=tokenizer.morphs,
             lower=True,
             batch_first=True)
LABEL = Field(sequential=False,
             use_vocab=False,
             preprocessing=lambda x: int(x),
             batch_first=True,
              is_target=True)
ID = Field(sequential=False,
          use_vocab=False,
          is_target=False)

NameError: ignored

### 2. 데이터셋 만들기

In [0]:
from torchtext.data import TabularDataset

dataset = TabularDataset(path='./nsmc/ratings.txt',
                        format='tsv',
                        fields=[('id', ID), ('text', TEXT), ('label', LABEL)],
                        skip_header=True)

### 3. 단어장 만들기

In [0]:
TEXT.build_vocab(dataset)
print('Total vocabulary: {}'.format(len(TEXT.vocab)))
print('Token for "<unk>": {}'.format(TEXT.vocab.stoi['<unk>']))
print('Token for "<pad>": {}'.format(TEXT.vocab.stoi['<pad>']))

Total vocabulary: 60825
Token for "<unk>": 0
Token for "<pad>": 1


### 4. 데이터로더 설정

In [0]:
from torchtext.data import Iterator

device = "cuda" if torch.cuda.is_available() else "cpu"
data_loader = Iterator(dataset=dataset,
                      batch_size=3,
                      device=device)

for batch in data_loader:
    break

print(batch.text.size(), batch.label.size())
print(batch.text)

torch.Size([3, 17]) torch.Size([3])
tensor([[  66,    2,  304,  124,  176,  262,   24,    4,    5,    2,  397,   20,
            9,   32,   27,    6,    2],
        [ 342,    2,  294,   69,  374,   45,  138,   11, 1088, 1311,    1,    1,
            1,    1,    1,    1,    1],
        [  87,   73,  827,   10, 9230,   19,  183,   57,   22,    9,  339,   75,
           56,   25,    6,    1,    1]])


## 코사인유사도

In [0]:
import torch

x1 = torch.FloatTensor([1, 2, 3, 4])
x2 = torch.FloatTensor([1, 2, 3, 5])
x3 = torch.FloatTensor([1, 4, 2, 1])

print(torch.cosine_similarity(x1, x2, dim=0))
print(torch.cosine_similarity(x1, x3, dim=0))

### 원핫인코딩과 코사인유사도

In [0]:
word1 = torch.FloatTensor([0, 0, 1, 0, 0, 0, 0, 0])
word2 = torch.FloatTensor([0, 0, 0, 0, 1, 0, 0, 0])
print(torch.cosine_similarity(word1, word2, dim=0))

## PyTorch로 사용하는 분산표상방식

In [0]:
tokens = "We are going to watch Avengers End Game".split()
new_sent = "We are Avengers".split()
vocab = {tkn: i for i, tkn in enumerate(tokens)}

embedding = torch.FloatTensor([[ 1, 5, 7],
                               [2, 1, 8],
                               [2, 6, 2],
                               [1, 6, 9],
                               [4, 5, 9],
                               [3, 2, 1],
                               [0, 9, 5],
                               [2, 3, 9]])

#새로운 문장 토큰들의 빈도표에 해당하는 인덱스
idxes = torch.LongTensor([vocab[tkn] for tkn in new_sent]) #인덱스는 항상 Long Tensor (정수형)

import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                               embedding_dim=3,
                               _weight=embedding)
print(embedding_layer(idxes))

## Word2Vec을 사용한 임베딩 행렬

Colab에 gensim 설치하기

In [0]:
! pip install -U gensim

In [0]:
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec

In [0]:
mecab = Mecab()
tokenizer = lambda x: ["/".join(y) for y in mecab.pos(x)]

f = open("/content/ratings.txt")
raw_data = f.read().splitlines()[1: ]
f.close()
data = [line.split("\t")[1] for line in raw_data]
data = [tokenizer(sent) for sent in data]

model = Word2Vec(sentences=data, size=100, window=5, min_count=3, sg=1)
#sentences: 문장 데이터. 하나의 list 속에 토큰화된 문장 list의 형태
#size: 임베딩 벡터의 크기
#window: skip-gram/CBoW 윈도우 크기
#min_count: 최소 등장 횟수
#sg: 1 if skip-gram; 0 if CBoW

print("단어 임베딩 행렬 크기 : ", model.wv.vectors.shape)
#학습한 모델 저장
model.wv.save_word2vec_format("/content/word2vec.pt")


### 학습한 모델을 사용해 코사인 유사도 구하기

In [0]:
sim1 = model.wv.similarity(*tokenizer("배우 여배우"))
sim2 = model.wv.similarity(*tokenizer("배우 교수"))
print("similarity(배우, 여배우) {:.2f}".format(sim1))
print("similarity(배우, 교수) {:.2f}".format(sim2))

In [0]:
sim3 = model.wv.most_similar(tokenizer("스토리"), topn=5)
for t, s in sim3:
  print("{} = {:.2f}".format(t, s))

In [0]:
sim4 = model.wv.most_similar(positive=tokenizer("여배우"), 
                             negative=tokenizer("남자"),
                             topn=1)
print(sim4)

# 순환 신경망

> 3.2.4 장에 해당하는 코드

In [0]:
# 코드 3-35

import torch
import torch.nn as nn

class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=True):
        super(CustomRNN, self).__init__()
        # 선형결합에 사용할 학습가능한 매개변수를 생성한다.
        self.weight_xh, self.weight_hh, self.bias = \
            self.init_weight(input_size, hidden_size)
        # 필요한 정보를 저장한다
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        
    def forward(self, inputs):
        """
        rnn_cell 을 구동하기 위해 inputs 의 크기 (T, B, E) 형태가 되어야 한다.
         - T: 시퀀스 총 길이
         - B: 미니배치크기
         - E: 입력층 크기 
        """
        if self.batch_first:
            # 첫번째 차원이 미니배치 크기인 경우 전치연산으로 바꿔준다.
            inputs = inputs.transpose(0, 1)
        seqlen, batch_size, _ = inputs.size()
        # 0 time-step 에서 은닉층 값을 0으로 초기화 시킨다
        hidden = self.init_hidden(batch_size, self.hidden_size)
        # output에 은닉층의 출력값을 저장한다.
        output = []
        # 시퀀스의 총 길이만큼 순방향전파를 진행한다.
        for i in range(seqlen):
            hidden = self.rnn_cell(inputs[i], hidden)
            output.append(hidden)
        output = torch.stack(output)
        if self.batch_first:
            output = output.transpose(0, 1)
        # 모든 타임스텝의 은닉층 출력값과 마지막 타임 스텝의 은닉층 출력값을 각각 반환한다.
        return output, hidden
    
    def rnn_cell(self, x, h):
        """RNN Cell"""
        h = x.mm(self.weight_xh.t()) + h.mm(self.weight_hh.t()) + self.bias
        return torch.tanh(h)
    
    def init_hidden(self, batch_size, hidden_size):
        """0 타임스텝에서 은닉층의 초기화"""
        return torch.zeros(batch_size, hidden_size)
    
    def init_weight(self, input_size, hidden_size):
        """rnn_cell 의 선형결합을 위한 초기값"""
        weight_xh = torch.randn(hidden_size, input_size).requires_grad_()
        weight_hh = torch.randn(hidden_size, hidden_size).requires_grad_()
        bias = torch.zeros(1, hidden_size).requires_grad_()
        return weight_xh, weight_hh, bias

In [0]:
# 코드 3-36

# RNN 호출 방법

rnn_layer = nn.RNN(input_size=5, hidden_size=10, batch_first=True)

# LSTM 호출 방법
#rnn_layer = nn.LSTM(input_size=5, hidden_size=10, batch_first=True) 
#print(rnn_layer)

# GRU 호출 방법
#rnn_layer = nn.GRU(input_size=5, hidden_size=10, batch_first=True)
#print(rnn_layer)

In [0]:
# 코드 3-37

import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(70)

sentence = "We are going to watch Advengers End Game".split()
vocab = {tkn: i for i, tkn in enumerate(sentence, 1)}  # 단어리스트 생성
vocab['<unk>'] = 0
# 수치화된 데이터를 단어로 바꾸기 위한 사전
rev_vocab = {v: k for k, v in vocab.items()}
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [rev_vocab.get(x) for x in y]  

def construct_data(sentence, vocab):
    """
    (input, target) 쌍으로 데이터를 생성한다.
    - 최종 형태(수치화된 단어 쌍):
     [(We,are) ,(are,going), (going,to), (to,watch), 
        (watch,Avengers), (Avengers,End), (End,Game)]
    """
    numericalize = lambda x: vocab.get(x) if vocab.get(x) is not None else 0
    totensor = lambda x: torch.LongTensor(x)
    idxes = [numericalize(token) for token in sentence]
    x, t = idxes[:-1], idxes[1:]
    return totensor(x).unsqueeze(0), totensor(t).unsqueeze(0)

class Net(nn.Module):
    """예제 문장을 출력하는 모델"""
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        """
        vocab_size: 단어장의 크기
        input_size: 임베딩 크기 = RNN의 입력층 크기
        hidden_size: RNN의 은닉층 크기
        """
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, 
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size,
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        """
        텐서 크기 변화 위한 문자 설명 
         - V: 단어장 크기
         - T: 시퀀스 총 길이
         - B: 미니배치 크기
         - E: 임베딩 크기 = RNN의 입력층 크기 
         - D: RNN 은닉층 크기
        """
        # 1. embedding 층을 총과해서 분산 표상 방식으로 단어를 표현
        # 크기변화: (B, T) > (B, T, D)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (B, T, D) > output (B, T, D), hidden (1, B, D)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (B, T, D) > (B, T, V)
        output = self.linear(output)
        # 4. 모델 출력: 
        # 크기변화: (B, T, V) > (B*T, V)
        return output.view(-1, output.size(2))
        
# -----------------------------------------
# 데이터 생성
x, t = construct_data(sentence, vocab)

# 모델 생성을 위한 하이퍼 파라미터 설정
vocab_size = len(vocab)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다
input_size = 5  # 임베딩된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss()
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

# 훈련 시작
for step in range(151):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(x)
    # 손실값 계산
    loss = loss_function(output, t.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 10 == 0:
        print("[{:02d}/151] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["We"] + decode(pred)))
        print()

# Deep RNN 과 Bidirectional RNN

> 3.2.5 장에 해당하는 코드

## Stacked RNN

In [0]:
# 코드 3-38

import torch
import torch.nn as nn

def is_equal(x, y):
    """두 텐서가 일치하는지 확인하는 함수"""
    return bool(x.eq(y).eq(0).sum() == 0)

# T: 시퀀스 총 길이
# B: 미니배치 크기
# E: RNN의 입력층 크기 
# D: RNN 은닉층 크기
# K: RNN 층의 개수(은닉층 깊이)

# 입력크기 (B, T, E) = (4, 7, 5)
sample_input = torch.randn(4, 7, 5)

# 층이 3개인 Stacked RNN 정의
rnn_layer = nn.RNN(input_size=5, 
                   hidden_size=4, 
                   num_layers=3,
                   batch_first=True)
print(rnn_layer)

output, hidden = rnn_layer(sample_input)
# output 크기: (B, T, D)
print("Output Size {}".format(output.size()))
# hidden 크기: (K, B, D)
print("Hidden Size {}".format(hidden.size()))

# 마지막 타임스텝의 K층 출력값(output[:, -1, :])과 
# 은닉층의 마지막 층 값(hidden[-1])이 일치하는지 확인
print("output[:, -1, :] == hidden[-1]? {}".format(
    is_equal(output[:, -1, :], hidden[-1])))

## Bidirectional RNN

In [0]:
# 코드 3-39

import torch
import torch.nn as nn

def is_equal(x, y):
    """두 텐서가 일치하는지 확인하는 함수"""
    return bool(x.eq(y).eq(0).sum() == 0)

# T: 시퀀스 총 길이
# B: 미니배치 크기
# E: RNN의 입력층 크기 
# D: RNN 은닉층 크기
# K: RNN 층의 개수(은닉층 깊이)

# 입력크기 (B, T, E) = (4, 7, 5)
sample_input = torch.randn(4, 7, 5)

# 층이 3개인 Bidirectional RNN 정의
rnn_layer = nn.RNN(input_size=5, 
                   hidden_size=4, 
                   num_layers=3,
                   batch_first=True,
                   bidirectional=True)
print(rnn_layer)

output, hidden = rnn_layer(sample_input)
# output 크기: (B, T, 2*D)
print("Output Size {}".format(output.size()))
# hidden 크기: (2*K, B, D)
print("Hidden Size {}".format(hidden.size()))

# 첫번째 미니배치에서, output의 마지막 타임스텝의 정방향 은닉값과 
# hidden의 마지막층 정방향 은닉값이 일치하는 지 확인
print("output[0, -1, :-4] == hidden[-2, 0, :]? {}".format(
    is_equal(output[0, -1, :-4], hidden[-2, 0, :])))

# 첫번째 미니배치에서, output의 첫번째 타임스텝의 역방향 은닉값과 
# hidden의 마지막층 역방향 은닉값이 일치하는 지 확인
print("output[0, 0, -4:] == hidden[-1, 0, :]? {}".format(
    is_equal(output[0, 0, -4:], hidden[-1, 0, :])))